In [43]:
import pandas as pd
from  tqdm import tqdm


In [44]:
#label_62= pd.read_excel('./label/62_label.xlsx',index_col=0)
label_66 =  pd.read_excel('./label/66_label.xlsx',index_col=0)

In [45]:
search_before = 60 *20 # 교체 전 몇 분, 초 단위로 입력

# 건들 X
def labeling(df_total:pd.DataFrame, search_before:int):
  idx = df_total["개수"].notnull()
  base_datetime = df_total[idx]["time"]
  base_datetime_list = list(base_datetime)
  df_total["이상치 라벨링"] = False

  for i in tqdm(base_datetime_list):
    base_datetime = pd.to_datetime(i)
    datetime_before = base_datetime - pd.Timedelta(seconds=search_before)

    df_total["이상치 라벨링"] = df_total["이상치 라벨링"] | ((df_total["time"] < i) & (df_total["time"] >= i - pd.Timedelta(seconds=search_before)))
    df_total["이상치 라벨링"] = df_total["이상치 라벨링"].astype(int)

  df_total.drop(['툴코드', '개수', 'mix', '제품코드', '규격'],axis=1,inplace=True)
  df_total.rename(columns = {'파생변수':'Derived Variable','이상치 라벨링':'label'},inplace=True)
  df_total.rename(columns={'time':'Time'},inplace=True)
  return df_total

In [46]:
label_66.columns

Index(['time', 'Active_Power_W1', 'Active_Power_W2', 'Active_Power_W3',
       'Apparent_Power_Va1', 'Apparent_Power_Va2', 'Apparent_Power_Va3',
       'Line_Current_L1', 'Line_Current_L2', 'Line_Current_L3',
       'Line_Voltage_V12', 'Line_Voltage_V23', 'Line_Voltage_V31', 'x', 'y',
       'z', '툴코드', '개수', 'mix', '제품코드', '규격', '파생변수', '이상치 라벨링'],
      dtype='object')

In [47]:
#label_62 =labeling(label_62,search_before)
label_66 =labeling(label_66,search_before)

100%|██████████| 29/29 [00:00<00:00, 49.64it/s]

In [28]:
#label_62['label'].value_counts()

In [48]:
x = pd.read_csv('./signal_xyz/x.csv')
x=x[['2','3','14','Time']]
x.columns = ['x_2', 'x_3','x_14', 'Time']
x['Time'] = pd.to_datetime(x['Time'])


y = pd.read_csv('./signal_xyz/y.csv')
y=y[['2','3','14','Time']]
y.columns = ['y_2', 'y_3','y_14', 'Time']
y['Time'] = pd.to_datetime(y['Time'])



z = pd.read_csv('./signal_xyz/z.csv')
z=z[['0','17','Time']]
z.columns = ['z_0', 'z_17', 'Time']
z['Time'] = pd.to_datetime(z['Time'])

In [30]:
def merge_dataframes_on_time(df1, df2):
    # 'Time' 열을 기준으로 두 데이터프레임을 합병
    # how='inner'는 두 데이터프레임에 모두 존재하는 'Time' 값에 대해서만 합병을 수행
    merged_df = pd.merge(df1, df2, on='Time', how='inner')
    return merged_df


In [31]:
#label_62= merge_dataframes_on_time(label_62,z)
label_66= merge_dataframes_on_time(label_66,x)
label_66= merge_dataframes_on_time(label_66,y)
label_66= merge_dataframes_on_time(label_66,z)

In [32]:
#label_62.to_csv('./label/new_label_62.csv',index=False)
label_66.to_csv('./label/new_label_66.csv',index=False)